In [3]:
%matplotlib inline

import os 
import json 

import boto3
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

COLOR_PALETTE = ['#648FFF', '#785EF0', '#DC267F', '#FE6100', '#FFB000']

In [25]:
s3_client = boto3.client('s3')
bucket = 'mimic-log-odds-models'
version = 19
prefix = f'movement-model-m3-a3-v{version}'

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)

configs = {}
if 'Contents' in response:
    for obj in response['Contents']:
        if obj['Key'].endswith('config.json'):
            config = json.loads(s3_client.get_object(Bucket=bucket, Key=obj['Key'])['Body'].read().decode('utf-8'))
            if 'model' in config:
                configs[config['run_id']] = config['model']
else:
    print("No objects found.")

rows = []
for run_id, config in configs.items():
    if 'num_layers' in config:
        rows.append({
            'run_id': run_id,
            'num_layers': config['num_layers'],
            'neurons': int(config['layers'][0][1:]),
            'dropout': 0.0 if not config['layers'][1].startswith('Dropout') else float(config['layers'][1][-1])/10
        })

configs_data = pd.DataFrame(rows)

results = (
    read_data_w_cache(f'select * from movement_model_experiment_m3_a3_v{version}')
    .sort_values(['run_id', 'epoch'], ascending=True)
)
results = results[['run_id', 'loss', 'val_loss', 'epoch']].merge(configs_data)
results.head()

,run_id,loss,val_loss,epoch,num_layers,neurons,dropout
0,039279bf762ed03a57e8d5b7f1fd4773a3383c8f522974...,0.310909,0.163783,1,4,16,0.2
1,039279bf762ed03a57e8d5b7f1fd4773a3383c8f522974...,0.200528,0.160396,2,4,16,0.2
2,039279bf762ed03a57e8d5b7f1fd4773a3383c8f522974...,0.192663,0.161556,3,4,16,0.2
3,039279bf762ed03a57e8d5b7f1fd4773a3383c8f522974...,0.189787,0.159795,4,4,16,0.2
4,039279bf762ed03a57e8d5b7f1fd4773a3383c8f522974...,0.188218,0.160977,5,4,16,0.2


In [26]:
final = results.groupby('run_id')[['val_loss', 'loss', 'epoch', 'neurons', 'dropout', 'num_layers']].last().reset_index().sort_values('val_loss', ascending=True)
final.head()

,run_id,val_loss,loss,epoch,neurons,dropout,num_layers
7,2ae8a5e9afc4bc7ca4f056a0305a8b50b202da32425e47...,0.151922,0.172620,50,16,0.2,3
52,fc1fe40a84007c459412a7c89b5ed821a5275db7f00ff0...,0.152372,0.164071,50,24,0.2,3
24,7c1f29926396533fa5a08edf3f7047632582e3c711f852...,0.152857,0.176076,50,24,0.3,3
4,1e002bd82efa3d9b03c1b1dd2bf70e237e39b1f7cb6232...,0.152954,0.165619,50,24,0.2,3
29,87094dfd3e017e60527049a2850e014291618cd4d7d055...,0.154094,0.165032,50,24,0.2,4


In [27]:
df = final.groupby(['neurons', 'dropout', 'num_layers'])[['val_loss', 'loss']].min().reset_index()
df['val_loss'].max() - df['val_loss'].min()

0.009944140911102295

In [28]:
run_id = final['run_id'].values[0]
run_id

'2ae8a5e9afc4bc7ca4f056a0305a8b50b202da32425e47ee595b53f143b3da5e'

In [33]:
px.line(
    results[results['run_id'] == run_id], x='epoch', y='val_loss'
)